In [11]:
import numpy as np, pandas as pd
import sqlite3 as sql
import os
import seaborn as sns
import plotly.graph_objs as go
from dotenv import load_dotenv
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import db_info, db_table_samples, select_statement_to_df, get_from_table

In [12]:
load_dotenv()

True

In [13]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 99)

In [14]:
def top_n_rated_timeframe(top_n, diary_date_from, diary_date_to, min_film_year=1800, max_film_year=2099, first_time_watch=True, film_genre=None, bottom=False):
    
    select_statement_a = """
    
    SELECT

         a.FILM_ID
        ,b.FILM_TITLE
        ,d.FILM_YEAR
        ,c.FILM_RATING_SCALED
        ,a.WATCH_DATE
	    ,e.FILM_GENRE
        
    FROM DIARY a
    LEFT JOIN FILM_TITLE b
    ON a.FILM_ID = b.FILM_ID
    LEFT JOIN PERSONAL_RATING c
    ON a.FILM_ID = c.FILM_ID
    LEFT JOIN FILM_YEAR d
    ON a.FILM_ID = d.FILM_ID
    LEFT JOIN FILM_GENRE e
    ON a.FILM_ID = e.FILM_ID

    WHERE a.WATCH_DATE >= "{}"
    AND a.WATCH_DATE <= "{}"
    AND a.IS_NARRATIVE_FEATURE = 1
    AND a.FIRST_TIME_WATCH = {}
    AND d.FILM_YEAR >= {}
    AND d.FILM_YEAR <= {}
    """.format(diary_date_from, diary_date_to, first_time_watch, min_film_year, max_film_year)

    if film_genre:
        select_statement_b = """
        AND e.FILM_GENRE LIKE "{}"
        """.format(film_genre)
    else:
        select_statement_b = ""
    if bottom:
        sort_order = 'ASC'
    else:
        sort_order = 'DESC'
    select_statement_c = """
    ORDER BY c.FILM_RATING_SCALED {}
    LIMIT {}
    """.format(sort_order, top_n)
    select_statement = select_statement_a + select_statement_b + select_statement_c
    return select_statement_to_df(select_statement)

In [16]:
top_n_rated_timeframe(top_n=50, diary_date_from='2025-01-01', diary_date_to='2025-12-31', min_film_year=1900, max_film_year=2023, first_time_watch=True, bottom=False)

,FILM_ID,FILM_TITLE,FILM_YEAR,FILM_RATING_SCALED,WATCH_DATE,FILM_GENRE
0,f_0tFwE,Lamb,2021,3.548982,2025-01-01,fantasy
